In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, CSVLogger

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPoolingV2

import numpy as np
import sklearn as sk

Using TensorFlow backend.


In [3]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=10000, sparse=False, remove_short_documents=True, verbose=False)

In [4]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
batch_size = 16
num_classes = Y_train.shape[1]

In [5]:
print(X_train.shape)
print(X_test.shape)
print(batch_size, num_classes)

(10168, 10000, 1)
(7071, 10000, 1)
16 20


In [6]:
%%time
METRIC = 'cosine'#'euclidean'
distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

# enforce exact zero
for k in xrange(distances.shape[0]):
  distances[k,k] = 0.

CPU times: user 426 ms, sys: 251 ms, total: 677 ms
Wall time: 1.34 s


In [7]:
%%time
# max normalize
#distances /= distances.max()
distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

# use tricube kernel (becaause of flatness around 0)
probabilities = (1. - np.abs(distances) ** 3) ** 3

# remove auto connections (which are taken anyway in LRF)
for k in xrange(probabilities.shape[0]):
  probabilities[k,k] = 0.

# normalize proba
probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))

CPU times: user 11.2 s, sys: 125 ms, total: 11.3 s
Wall time: 11.5 s


In [8]:
%%time
model = Sequential()
model.add(MonteCarloLRF(probabilities, LRF_size=16, filters=64, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(SeparableMonteCarloMaxPoolingV2(LRF_size=4, new_size=2500))
#model.add(MonteCarloLRF(probabilities, LRF_size=16, filters=64, activation='relu',
#                       input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Conv1D(16, kernel_size=1, activation='relu',
                 padding='same',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
monte_carlo_lrf_1 (MonteCarl (None, 10000, 64)         1088      
_________________________________________________________________
separable_monte_carlo_max_po (None, 2500, 64)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2500, 16)          1040      
_________________________________________________________________
dropout_1 (Dropout)          (None, 2500, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                800020    
Total params: 802,148
Trainable params: 802,148
Non-trainable params: 0
_________________________________________________________________
CPU 

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
csv = CSVLogger('mcNet_top10k_temptative_2_log.csv')

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    callbacks=[early_stopper, csv],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/20
10168/10168 [==============================] - 190s - loss: 2.3413 - acc: 0.3497 - val_loss: 1.6485 - val_acc: 0.5641
Epoch 2/20
10168/10168 [==============================] - 188s - loss: 0.8766 - acc: 0.7784 - val_loss: 1.2983 - val_acc: 0.6529
Epoch 3/20
10168/10168 [==============================] - 188s - loss: 0.4053 - acc: 0.8934 - val_loss: 1.4201 - val_acc: 0.6429
Epoch 4/20
10168/10168 [==============================] - 188s - loss: 0.2089 - acc: 0.9471 - val_loss: 1.4192 - val_acc: 0.6637
Epoch 5/20
10168/10168 [==============================] - 188s - loss: 0.1160 - acc: 0.9739 - val_loss: 1.5305 - val_acc: 0.6651
Test loss: 1.53053414055
Test accuracy: 0.665111016863


In [10]:
%%bash
tail mcNet_top10k_temptative_2_log.csv

epoch,acc,loss,val_acc,val_loss
0,0.34972462627852086,2.3413197327372974,0.56413520016371488,1.648478078430806
1,0.77842250196695517,0.87663142084137458,0.65294866372395599,1.2982822778184147
2,0.89339103068450043,0.4052770104490801,0.64290765102775171,1.4201334336238609
3,0.94708890637293475,0.20887747781979907,0.6636967898730155,1.4192432149972862
4,0.97393784421715179,0.1159904236327505,0.66511101699789177,1.5305341346785499
